In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
from datetime import date
import sys
import os
pd.set_option('display.max_row',200)

def printDayOfTheWeek (year, month, day):
    dayOfTheWeek = ["월요일", "화요일", "수요일", "목요일", "금요일", "토요일", "일요일"]
    return dayOfTheWeek[date(year, month, day).weekday()]

In [2]:
os.chdir('C:\\Users\\alswo\\Desktop\\식파마\\BasketAnalysis\\data\\상인점\\')
os.getcwd()

'C:\\Users\\alswo\\Desktop\\식파마\\BasketAnalysis\\data\\상인점'

In [3]:
df = pd.read_excel('상인점_전처리완료_2019_11.xlsx')
del df['Unnamed: 0']
df

,판매일자,영수증번호,판매시간,영수증별총매출,현금,카드,key,구분,메뉴코드,메뉴명,메뉴가격,메뉴수량,메뉴가격*수량
0,2019-11-01,01-0001,09:28:42,8500,0,8500,r1,판매,0002,순대국밥,7000,1,7000
1,2019-11-01,01-0001,09:28:42,8500,0,8500,r1,판매,0047,음료수,1500,1,1500
2,2019-11-01,01-0002,09:35:02,25500,0,25500,r2,판매,0002,순대국밥,7000,2,14000
3,2019-11-01,01-0002,09:35:02,25500,0,25500,r2,판매,0005,모둠국밥,7500,1,7500
4,2019-11-01,01-0002,09:35:02,25500,0,25500,r2,판매,0043,소주,4000,1,4000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6375,2019-11-30,01-0133,09:06:03,42000,42000,0,r3354,판매,0043,소주,4000,4,16000
6376,2019-11-30,01-0133,09:06:03,42000,42000,0,r3354,판매,0047,음료수,1500,4,6000
6377,2019-11-30,01-0134,09:09:57,6500,0,6500,r3355,판매,0001,돼지국밥,6500,1,6500
6378,2019-11-30,01-0135,09:28:13,26000,0,26000,r3356,판매,0001,돼지국밥,6500,4,26000


In [4]:
df['요일'] = df['판매일자'].apply(lambda x: printDayOfTheWeek(int(x[:4]),int(x[5:7]),int(x[8:])))
df

,판매일자,영수증번호,판매시간,영수증별총매출,현금,카드,key,구분,메뉴코드,메뉴명,메뉴가격,메뉴수량,메뉴가격*수량,요일
0,2019-11-01,01-0001,09:28:42,8500,0,8500,r1,판매,0002,순대국밥,7000,1,7000,금요일
1,2019-11-01,01-0001,09:28:42,8500,0,8500,r1,판매,0047,음료수,1500,1,1500,금요일
2,2019-11-01,01-0002,09:35:02,25500,0,25500,r2,판매,0002,순대국밥,7000,2,14000,금요일
3,2019-11-01,01-0002,09:35:02,25500,0,25500,r2,판매,0005,모둠국밥,7500,1,7500,금요일
4,2019-11-01,01-0002,09:35:02,25500,0,25500,r2,판매,0043,소주,4000,1,4000,금요일
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6375,2019-11-30,01-0133,09:06:03,42000,42000,0,r3354,판매,0043,소주,4000,4,16000,토요일
6376,2019-11-30,01-0133,09:06:03,42000,42000,0,r3354,판매,0047,음료수,1500,4,6000,토요일
6377,2019-11-30,01-0134,09:09:57,6500,0,6500,r3355,판매,0001,돼지국밥,6500,1,6500,토요일
6378,2019-11-30,01-0135,09:28:13,26000,0,26000,r3356,판매,0001,돼지국밥,6500,4,26000,토요일


In [5]:
# 3시간마다 
df['시간분할'] = df['판매시간'].apply(lambda x: int((int(x[:2])-(int(x[:2])%3))/3))
df

,판매일자,영수증번호,판매시간,영수증별총매출,현금,카드,key,구분,메뉴코드,메뉴명,메뉴가격,메뉴수량,메뉴가격*수량,요일,시간분할
0,2019-11-01,01-0001,09:28:42,8500,0,8500,r1,판매,0002,순대국밥,7000,1,7000,금요일,3
1,2019-11-01,01-0001,09:28:42,8500,0,8500,r1,판매,0047,음료수,1500,1,1500,금요일,3
2,2019-11-01,01-0002,09:35:02,25500,0,25500,r2,판매,0002,순대국밥,7000,2,14000,금요일,3
3,2019-11-01,01-0002,09:35:02,25500,0,25500,r2,판매,0005,모둠국밥,7500,1,7500,금요일,3
4,2019-11-01,01-0002,09:35:02,25500,0,25500,r2,판매,0043,소주,4000,1,4000,금요일,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6375,2019-11-30,01-0133,09:06:03,42000,42000,0,r3354,판매,0043,소주,4000,4,16000,토요일,3
6376,2019-11-30,01-0133,09:06:03,42000,42000,0,r3354,판매,0047,음료수,1500,4,6000,토요일,3
6377,2019-11-30,01-0134,09:09:57,6500,0,6500,r3355,판매,0001,돼지국밥,6500,1,6500,토요일,3
6378,2019-11-30,01-0135,09:28:13,26000,0,26000,r3356,판매,0001,돼지국밥,6500,4,26000,토요일,3


In [6]:
total_income = 0
for u in df['메뉴가격*수량'] :
    total_income += u
total_income

61266500

In [7]:
df_2 = df[['판매일자','판매시간','메뉴코드','영수증번호','메뉴명','요일','시간분할']]
df_2['메뉴명']=df_2['메뉴명'].apply(lambda x: '소주' if x=='참소주' or x=='참이슬' else 
                             ('맥주' if x=='카스' or x=='하이트' else x))

df_2['계절']=df_2['판매일자'].apply(lambda x: 'spring' if 3<=int(x[5:7])<6 else
                             ('summer' if 6<=int(x[5:7])<9 else
                             ('fall' if 9<=int(x[5:7])<12 else 'winter')))
df_2.head()

C:\Users\alswo\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\alswo\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,판매일자,판매시간,메뉴코드,영수증번호,메뉴명,요일,시간분할,계절
0,2019-11-01,09:28:42,0002,01-0001,순대국밥,금요일,3,fall
1,2019-11-01,09:28:42,0047,01-0001,음료수,금요일,3,fall
2,2019-11-01,09:35:02,0002,01-0002,순대국밥,금요일,3,fall
3,2019-11-01,09:35:02,0005,01-0002,모둠국밥,금요일,3,fall
4,2019-11-01,09:35:02,0043,01-0002,소주,금요일,3,fall


In [9]:
cnt=1

pre_date = df_2.iloc[0]['판매일자']
pre_num = df_2.iloc[0]['영수증번호']
pre_week = df_2.iloc[0]['요일']
pre_part = df_2.iloc[0]['시간분할']

drop_list=[]

for i,idx in enumerate(df_2.index):
    temp_list = []
    
    time = df_2.iloc[i]['판매시간']
    df_2.loc[idx,'판매시간.1']=time[:2]
    
    if i==0:
        continue

    if (df_2.iloc[i]['판매일자']==pre_date) and (df_2.iloc[i]['영수증번호']==pre_num):
        cnt=cnt+1
    elif (pre_date!=df_2.iloc[i]['영수증번호']) or (pre_num!=df_2.iloc[i]['판매일자']):
        
        if cnt >= 2:
            # temp_list.append(df_2.loc[i-1]['메뉴명'])
            # temp_list=[]
            cnt = 1
            pass
        elif cnt < 2:
            drop_list.append(i-1)
            
        if i == (len(df_2)-1):
            drop_list.append(i)
            
        cnt = 1
        
    pre_date = df_2.iloc[i]['판매일자']
    pre_num = df_2.iloc[i]['영수증번호']

df_2

C:\Users\alswo\anaconda3\lib\site-packages\pandas\core\indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\alswo\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,판매일자,판매시간,메뉴코드,영수증번호,메뉴명,요일,시간분할,계절,판매시간.1
0,2019-11-01,09:28:42,0002,01-0001,순대국밥,금요일,3,fall,09
1,2019-11-01,09:28:42,0047,01-0001,음료수,금요일,3,fall,09
2,2019-11-01,09:35:02,0002,01-0002,순대국밥,금요일,3,fall,09
3,2019-11-01,09:35:02,0005,01-0002,모둠국밥,금요일,3,fall,09
4,2019-11-01,09:35:02,0043,01-0002,소주,금요일,3,fall,09
...,...,...,...,...,...,...,...,...,...
6375,2019-11-30,09:06:03,0043,01-0133,소주,토요일,3,fall,09
6376,2019-11-30,09:06:03,0047,01-0133,음료수,토요일,3,fall,09
6377,2019-11-30,09:09:57,0001,01-0134,돼지국밥,토요일,3,fall,09
6378,2019-11-30,09:28:13,0001,01-0135,돼지국밥,토요일,3,fall,09


In [10]:
df_2.loc[drop_list]

,판매일자,판매시간,메뉴코드,영수증번호,메뉴명,요일,시간분할,계절,판매시간.1
5,2019-11-01,10:48:50,0001,01-0003,돼지국밥,금요일,3,fall,10
6,2019-11-01,11:01:03,0001,01-0004,돼지국밥,금요일,3,fall,11
7,2019-11-01,11:21:28,0001,01-0005,돼지국밥,금요일,3,fall,11
10,2019-11-01,11:43:18,0006,01-0007,무보까정식,금요일,3,fall,11
11,2019-11-01,12:03:34,0010,01-0008,갈비시레기국밥,금요일,4,fall,12
...,...,...,...,...,...,...,...,...,...
6350,2019-11-30,06:36:27,0006,01-0123,무보까정식,토요일,2,fall,06
6354,2019-11-30,07:09:03,0005,01-0125,모둠국밥,토요일,2,fall,07
6377,2019-11-30,09:09:57,0001,01-0134,돼지국밥,토요일,3,fall,09
6378,2019-11-30,09:28:13,0001,01-0135,돼지국밥,토요일,3,fall,09


In [11]:
df_ac = df_2.drop(drop_list)
df_ac

,판매일자,판매시간,메뉴코드,영수증번호,메뉴명,요일,시간분할,계절,판매시간.1
0,2019-11-01,09:28:42,0002,01-0001,순대국밥,금요일,3,fall,09
1,2019-11-01,09:28:42,0047,01-0001,음료수,금요일,3,fall,09
2,2019-11-01,09:35:02,0002,01-0002,순대국밥,금요일,3,fall,09
3,2019-11-01,09:35:02,0005,01-0002,모둠국밥,금요일,3,fall,09
4,2019-11-01,09:35:02,0043,01-0002,소주,금요일,3,fall,09
...,...,...,...,...,...,...,...,...,...
6372,2019-11-30,09:04:03,0010,01-0132,갈비시레기국밥,토요일,3,fall,09
6373,2019-11-30,09:06:03,0001,01-0133,돼지국밥,토요일,3,fall,09
6374,2019-11-30,09:06:03,0002,01-0133,순대국밥,토요일,3,fall,09
6375,2019-11-30,09:06:03,0043,01-0133,소주,토요일,3,fall,09


In [12]:
pre_num=df_ac.iloc[0]['영수증번호']
pre_date=df_ac.iloc[0]['판매일자']

temp_list=[]
total_list=[]

for i,idx in enumerate(df_ac):
    temp_list.append(df_ac.iloc[i]['메뉴명'])
    
    if pre_num != df_ac.iloc[i]['영수증번호']:
        total_list.append(temp_list)
        temp_list=[]
        
    if i==(len(df_ac)-1):
        total_list.append(temp_list)

sum=0

for i in range(0,len(total_list)):
    sum+=len(total_list[i])
    
print(total_list)

[['순대국밥', '음료수', '순대국밥'], ['모둠국밥'], ['소주'], ['순대국밥'], ['짬뽕순두부국밥'], ['순대국밥'], ['공기밥']]


In [13]:
date_dict={}
time_dict={}
receipt_dict={}

temp_list=[]
total_list=[]

pre_date=df_ac.iloc[0]['판매일자']
pre_time=df_ac.iloc[0]['판매시간.1']
pre_receipt=df_ac.iloc[0]['영수증번호']

for i,idx in enumerate(df_ac.index):
    temp_list.append(df_ac.iloc[i]['메뉴명'])
    
    ## {영수증별:메뉴리스트} 딕셔너리 채우기 ##
    if pre_receipt!=df_ac.iloc[i]['영수증번호']:
#         if receipt_dict.get(pre_receipt):
#             print('already exist receipt!!')
#             pass
#         else:
#             receipt_dict[pre_receipt]=temp_list
        total_list.append(temp_list)
        temp_list=[]
            
    ## {시간별:영수증별} 딕셔너리 채우기 ##
    if pre_time!=df_ac.iloc[i]['판매시간']:
        if time_dict.get(pre_time):
            print(i,',',pre_time,',',df_ac.iloc[i]['판매시간'])
            print('already exist time!!')
            pass
        else:
            time_dict[pre_time]=total_list
#             time_dict[pre_time]=receipt_dict
#         receipt_dict={}
        total_list=[]
        
    ## {날짜별:시간별} 딕셔너리 채우기 ##
    if pre_date!=df_ac.iloc[i]['판매일자']:
        if date_dict.get(pre_date):
            print('already exist date!!')
            pass
        else:
            date_dict[pre_date]=time_dict
        time_dict={}
        
    if i==(len(df_ac)-1):
        receipt_dict[pre_receipt]=temp_list
        time_dict[pre_time]=receipt_dict
        date_dict[pre_date]=time_dict
      
    pre_date=df_ac.loc[idx]['판매일자']
    pre_time=df_ac.loc[idx]['판매시간']
    pre_receipt=df_ac.loc[idx]['영수증번호']

In [14]:
df_sorted_part=df_ac.sort_values(by=['계절','시간분할','판매일자','영수증번호'])
df_sorted_part

,판매일자,판매시간,메뉴코드,영수증번호,메뉴명,요일,시간분할,계절,판매시간.1
139,2019-11-01,00:45:03,0006,01-0083,무보까정식,금요일,0,fall,00
140,2019-11-01,00:45:03,0018,01-0083,공기밥,금요일,0,fall,00
141,2019-11-01,01:01:09,0001,01-0084,돼지국밥,금요일,0,fall,01
142,2019-11-01,01:01:09,0047,01-0084,음료수,금요일,0,fall,01
144,2019-11-01,01:19:15,0044,01-0086,맥주,금요일,0,fall,01
...,...,...,...,...,...,...,...,...,...
6266,2019-11-30,23:23:54,0043,01-0090,소주,토요일,7,fall,23
6267,2019-11-30,23:32:50,0004,01-0091,얼큰이국밥,토요일,7,fall,23
6268,2019-11-30,23:32:50,0043,01-0091,소주,토요일,7,fall,23
6270,2019-11-30,23:59:44,0006,01-0093,무보까정식,토요일,7,fall,23


In [15]:
## 3시간 단위로 묶기

season_dict={}

part_dict={}

temp_list=[]
result_list=[]

pre_part=df_sorted_part.iloc[0]['시간분할']
pre_season=df_sorted_part.iloc[0]['계절']
pre_date=df_sorted_part.iloc[0]['판매일자']
pre_receipt=df_sorted_part.iloc[0]['영수증번호']

temp_list.append(df_sorted_part.iloc[0]['메뉴명'])

for i,idx in enumerate(df_sorted_part.index):
    
    cond0 = pre_season!=df_sorted_part.iloc[i]['계절']
    cond1 = pre_receipt!=df_sorted_part.iloc[i]['영수증번호']
    cond2 = pre_date!=df_sorted_part.iloc[i]['판매일자']
    cond3 = pre_part!=df_sorted_part.iloc[i]['시간분할']
    
    if cond1 or cond2 or cond3:
        result_list.append(temp_list)
        temp_list=[]
        
        if cond3:
            part_dict[pre_part]=result_list
            result_list=[]
            
    if i==(len(df_sorted_part)-1):
        part_dict[pre_part]=result_list
        
    if cond0 or i==(len(df_sorted_part)-1):
        season_dict[pre_season]=part_dict
        part_dict={}
            
    if i>0:
        temp_list.append(df_sorted_part.iloc[i]['메뉴명'])
                    
    pre_part=df_sorted_part.iloc[i]['시간분할']
    pre_date=df_sorted_part.iloc[i]['판매일자']
    pre_season=df_sorted_part.iloc[i]['계절']
    pre_receipt=df_sorted_part.iloc[i]['영수증번호']

In [16]:
season_dict

{'fall': {0: [['무보까정식', '공기밥'],
   ['돼지국밥', '음료수'],
   ['맥주', '음료수', '짬뽕순두부국밥', '떡갈비정식'],
   ['무보까정식', '소주'],
   ['돼지국밥', '얼큰이국밥', '소주', '짬뽕순두부국밥'],
   ['돼지국밥', '소주', '순대한접시'],
   ['무보까보쌈(중)', '소주', '맥주'],
   ['모둠국밥', '무보까정식', '공기밥'],
   ['돼지국밥', '소주', '음료수', '짬뽕순두부국밥'],
   ['무보까정식', '소주', '짬뽕순두부국밥'],
   ['돼지국밥', '내장국밥', '공기밥'],
   ['무보까정식', '곱배기', '수제떡갈비'],
   ['무보까정식', '소주'],
   ['돼지국밥', '공기밥'],
   ['돼지국밥', '순대국밥', '공기밥'],
   ['소주', '짬뽕순두부국밥'],
   ['순대국밥', '모둠국밥', '갈비시레기국밥', '맥주', '수제떡갈비'],
   ['내장국밥', '고기만두', '소주'],
   ['공기밥', '짬뽕순두부국밥'],
   ['돼지국밥', '순대국밥', '막걸리'],
   ['돼지국밥', '순대국밥', '소주', '짬뽕순두부국밥'],
   ['돼지국밥', '순대국밥', '공기밥'],
   ['돼지국밥', '모둠국밥', '짬뽕순두부국밥'],
   ['돼지국밥', '소주', '맥주'],
   ['돼지국밥', '공기밥', '짬뽕순두부국밥'],
   ['얼큰이국밥', '무보까정식', '음료수'],
   ['돼지국밥', '소주', '순대한접시'],
   ['공기밥', '떡갈비정식'],
   ['돼지국밥', '소주', '음료수', '짬뽕순두부국밥'],
   ['무보까정식', '소주', '음료수', '떡갈비정식'],
   ['음료수', '내장국밥', '시레기들깨국밥', '공기밥', '소주'],
   ['돼지국밥', '순대국밥'],
   ['돼지국밥', '공기밥', '맛보기순대', '떡갈비정식'],
   ['돼지국밥', '순대

In [17]:
df_week=df_ac.sort_values(by=['계절','요일','판매일자','영수증번호'])
df_week

,판매일자,판매시간,메뉴코드,영수증번호,메뉴명,요일,시간분할,계절,판매시간.1
0,2019-11-01,09:28:42,0002,01-0001,순대국밥,금요일,3,fall,09
1,2019-11-01,09:28:42,0047,01-0001,음료수,금요일,3,fall,09
2,2019-11-01,09:35:02,0002,01-0002,순대국밥,금요일,3,fall,09
3,2019-11-01,09:35:02,0005,01-0002,모둠국밥,금요일,3,fall,09
4,2019-11-01,09:35:02,0043,01-0002,소주,금요일,3,fall,09
...,...,...,...,...,...,...,...,...,...
5480,2019-11-26,06:12:34,0005,01-0086,모둠국밥,화요일,2,fall,06
5481,2019-11-26,06:12:34,0018,01-0086,공기밥,화요일,2,fall,06
5482,2019-11-26,06:12:34,0023,01-0086,무보까보쌈(중),화요일,2,fall,06
5483,2019-11-26,09:31:23,0002,01-0087,순대국밥,화요일,3,fall,09


In [18]:
## 요일별로 묶기

season_week_dict={}

week_dict={}

temp_list=[]
result_list=[]

pre_week=df_week.iloc[0]['요일']
pre_receipt=df_week.iloc[0]['영수증번호']
pre_date=df_week.iloc[0]['판매일자']
pre_season=df_week.iloc[0]['계절']

temp_list.append(df_week.iloc[0]['메뉴명'])

for i,idx in enumerate(df_week.index):
    
    cond0 = pre_season!=df_week.iloc[i]['계절']
    cond1 = pre_receipt!=df_week.iloc[i]['영수증번호']
    cond2 = pre_date!=df_week.iloc[i]['판매일자']
    cond3 = pre_week!=df_week.iloc[i]['요일']
    
    if cond1 or cond2 or cond3:
        result_list.append(temp_list)
        temp_list=[]
        
        if cond3:
            week_dict[pre_week]=result_list
            result_list=[]
            
    if i==(len(df_sorted_part)-1):
        week_dict[pre_week]=result_list
    
    if cond0 or i==(len(df_week)-1):
        season_week_dict[pre_season]=week_dict
        week_dict={}
    
    if i>0:
        temp_list.append(df_week.iloc[i]['메뉴명'])
            
    
#     if (pre_receipt!=df_week.iloc[i]['영수증번호']) or (pre_week!=df_week.iloc[i]['요일']):
#         result_list.append(temp_list)
#         temp_list=[]
    
#     if pre_week!=df_week.iloc[i]['요일']:
#         if week_dict.get(pre_week):
#             for elem_list in result_list:
#                 week_dict[pre_week].append(elem_list)
#         else:
#             if pre_receipt==df_week.iloc[i]['영수증번호']:
#                 result_list.append(temp_list)
#             week_dict[pre_week]=result_list
#         result_list=[]
#         temp_list=[]
            
#     if i==(len(df_ac)-1):
#         if week_dict.get(pre_week):
#             for elem_list in result_list:
#                 week_dict[pre_week].append(elem_list)
#         else:
#             result_list.append(temp_list)
#             week_dict[pre_week]=result_list
        
#     if cond0 or i==(len(df_week)-1):
#         season_week_dict[pre_season]=week_dict
#         week_dict={}
        
#     if i>0:
#         temp_list.append(df_week.iloc[i]['메뉴명'])
            
    pre_date=df_week.iloc[i]['판매일자']
    pre_week=df_week.iloc[i]['요일']
    pre_receipt=df_week.iloc[i]['영수증번호']
    pre_season=df_week.iloc[i]['계절']

In [19]:
season_week_dict

{'fall': {'금요일': [['순대국밥', '음료수'],
   ['순대국밥', '모둠국밥', '소주'],
   ['순대국밥', '짬뽕순두부국밥'],
   ['순대국밥', '공기밥'],
   ['시레기들깨국밥', '떡갈비정식'],
   ['돼지국밥', '맛보기순대'],
   ['얼큰이국밥', '모둠국밥'],
   ['돼지국밥', '순대국밥'],
   ['공기밥', '떡갈비정식'],
   ['시레기들깨국밥', '소주'],
   ['돼지국밥', '순대국밥', '공기밥'],
   ['순대국밥', '곱배기', '짬뽕순두부국밥'],
   ['돼지국밥', '소주'],
   ['돼지국밥', '내장국밥', '음료수'],
   ['돼지국밥', '얼큰이국밥'],
   ['돼지국밥', '무보까정식'],
   ['음료수', '모둠국밥', '시레기들깨국밥', '갈비시레기국밥', '맥주'],
   ['순대국밥', '얼큰이국밥'],
   ['돼지국밥', '소주'],
   ['공기밥', '매콤이소갈비찜(중)', '소주'],
   ['공기밥', '짬뽕순두부국밥'],
   ['순대국밥', '내장국밥', '소주'],
   ['모둠국밥', '소주'],
   ['돼지국밥', '얼큰이국밥', '소주'],
   ['돼지국밥', '떡갈비정식'],
   ['돼지국밥', '무보까정식'],
   ['시레기들깨국밥', '공기밥', '마늘보쌈(중)'],
   ['공기밥', '매콤이소갈비찜(중)'],
   ['내장국밥', '얼큰이국밥', '공기밥'],
   ['돼지국밥', '소주'],
   ['무보까정식', '돼지국밥(행사)'],
   ['순대국밥', '내장국밥', '얼큰이국밥', '소주'],
   ['돼지국밥', '소주'],
   ['돼지국밥', '소주'],
   ['돼지국밥', '순대국밥', '공기밥'],
   ['순대국밥', '공기밥', '짬뽕순두부국밥'],
   ['돼지국밥', '순대국밥', '공기밥', '짬뽕순두부국밥'],
   ['공기밥', '순대곱창전골(중)'],
   ['돼지국밥', '얼큰이국밥

In [20]:
day_list=['mon','tue','wed','thu','fri','sat','sun']
#season_list=['spring','summer','fall','winter']
season_list=['fall']

val_list=[]

te = TransactionEncoder()

for season in season_list:
    for day in day_list:

        week_day = '월요일'

        if day=='mon':
            week_day='월요일'
        elif day=='tue':
            week_day='화요일'
        elif day=='wed':
            week_day='수요일'
        elif day=='thu':
            week_day='목요일'
        elif day=='fri':
            week_day='금요일'
        elif day=='sat':
            week_day='토요일'
        else:
            week_day='일요일'

        te_result = te.fit(season_week_dict[season][week_day]).transform(season_week_dict[season][week_day])

        te_df = pd.DataFrame(te_result, columns=te.columns_)

        globals()['itemsets_{}_{}'.format(season[:3],day)]=apriori(te_df, min_support=0.006, use_colnames=True)

        val_list.append('itemsets_{}_{}'.format(season[:3],day))


In [21]:
association_day_val_list=[]

for val_name in val_list:
        
    globals()['result_day_{}'.format(val_name[-7:])] = association_rules(globals()[val_name],metric="lift",min_threshold=1)
    
    association_day_val_list.append('result_day_{}'.format(val_name[-7:]))

In [22]:
result_day_fal_mon

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(짬뽕순두부국밥),(갈비시레기국밥),0.091837,0.056122,0.015306,0.166667,2.969697,0.010152,1.132653
1,(갈비시레기국밥),(짬뽕순두부국밥),0.056122,0.091837,0.015306,0.272727,2.969697,0.010152,1.248724
2,(고기만두),(내장국밥),0.071429,0.081633,0.010204,0.142857,1.750000,0.004373,1.071429
3,(내장국밥),(고기만두),0.081633,0.071429,0.010204,0.125000,1.750000,0.004373,1.061224
4,(고기만두),(모둠국밥),0.071429,0.107143,0.020408,0.285714,2.666667,0.012755,1.250000
...,...,...,...,...,...,...,...,...,...
211,(무보까정식),"(짬뽕순두부국밥, 소주)",0.193878,0.015306,0.010204,0.052632,3.438596,0.007237,1.039399
212,"(순대국밥, 소주)",(음료수),0.061224,0.132653,0.010204,0.166667,1.256410,0.002082,1.040816
213,"(음료수, 소주)",(순대국밥),0.030612,0.250000,0.010204,0.333333,1.333333,0.002551,1.125000
214,(순대국밥),"(음료수, 소주)",0.250000,0.030612,0.010204,0.040816,1.333333,0.002551,1.010638


In [23]:
nov = result_day_fal_mon
nov.drop('leverage', axis = 1, inplace = True)
nov.drop('conviction', axis = 1, inplace = True)
nov.drop('antecedent support', axis = 1, inplace = True)
nov.drop('consequent support', axis = 1, inplace = True)
nov

,antecedents,consequents,support,confidence,lift
0,(짬뽕순두부국밥),(갈비시레기국밥),0.015306,0.166667,2.969697
1,(갈비시레기국밥),(짬뽕순두부국밥),0.015306,0.272727,2.969697
2,(고기만두),(내장국밥),0.010204,0.142857,1.750000
3,(내장국밥),(고기만두),0.010204,0.125000,1.750000
4,(고기만두),(모둠국밥),0.020408,0.285714,2.666667
...,...,...,...,...,...
211,(무보까정식),"(짬뽕순두부국밥, 소주)",0.010204,0.052632,3.438596
212,"(순대국밥, 소주)",(음료수),0.010204,0.166667,1.256410
213,"(음료수, 소주)",(순대국밥),0.010204,0.333333,1.333333
214,(순대국밥),"(음료수, 소주)",0.010204,0.040816,1.333333


In [68]:
nov['antecedents']

menu_name_list_1 = []

for q in nov['antecedents'] :
    de = list(q)
    menu_name_list_1.append(de)
menu_name_list_1 = [i[0:] for i in menu_name_list_1]
menu_name_list_1

[['짬뽕순두부국밥'],
 ['갈비시레기국밥'],
 ['고기만두'],
 ['내장국밥'],
 ['고기만두'],
 ['모둠국밥'],
 ['고기만두'],
 ['음료수'],
 ['돼지국밥'],
 ['곱배기'],
 ['공기밥'],
 ['내장국밥'],
 ['떡갈비정식'],
 ['공기밥'],
 ['마늘보쌈(중)'],
 ['공기밥'],
 ['매콤이소갈비찜(중)'],
 ['공기밥'],
 ['무보까보쌈(중)'],
 ['공기밥'],
 ['순대곱창전골(중)'],
 ['공기밥'],
 ['음료수'],
 ['공기밥'],
 ['돼지국밥'],
 ['내장국밥'],
 ['막걸리'],
 ['내장국밥'],
 ['맛보기순대'],
 ['내장국밥'],
 ['순대국밥'],
 ['내장국밥'],
 ['도가니탕'],
 ['무보까정식'],
 ['도가니탕'],
 ['소주'],
 ['돼지국밥'],
 ['떡갈비정식'],
 ['막걸리'],
 ['돼지국밥'],
 ['돼지국밥'],
 ['수육(중)'],
 ['순대국밥'],
 ['돼지국밥'],
 ['떡갈비정식'],
 ['소주'],
 ['음료수'],
 ['떡갈비정식'],
 ['마늘보쌈(중)'],
 ['소주'],
 ['순대국밥'],
 ['막걸리'],
 ['맛보기순대'],
 ['소주'],
 ['맥주'],
 ['무보까정식'],
 ['맥주'],
 ['소주'],
 ['순대국밥'],
 ['맥주'],
 ['소주'],
 ['모둠국밥'],
 ['얼큰이국밥'],
 ['모둠국밥'],
 ['소주'],
 ['무보까보쌈(대)'],
 ['무보까보쌈(중)'],
 ['소주'],
 ['음료수'],
 ['무보까정식'],
 ['짬뽕순두부국밥'],
 ['무보까정식'],
 ['소주'],
 ['수육(중)'],
 ['순대국밥'],
 ['수육(중)'],
 ['음료수'],
 ['수육(중)'],
 ['순대국밥'],
 ['음료수'],
 ['얼큰이국밥'],
 ['순대한접시'],
 ['얼큰이국밥'],
 ['음료수'],
 ['얼큰이국밥'],
 ['짬뽕순두부국밥'],
 ['고기만두', '돼지국밥'],
 ['고기만두', '공기밥'],

In [28]:
os.chdir('C:\\Users\\alswo\\Desktop\\식파마\\BasketAnalysis\\data\\상인점\\')
df_price = pd.read_csv('상인점_빈도수_가격.csv', encoding = 'cp949')
df_price

,메뉴,빈도수,가격
0,돼지국밥,176,6500
1,순대국밥,73,7000
2,무보까정식,69,9000
3,소주,63,4000
4,공기밥,53,1000
5,얼큰이국밥,31,7500
6,모둠국밥,29,7500
7,짬뽕순두부국밥,27,7500
8,음료수,27,1500
9,내장국밥,23,7000


In [29]:
pr_list_1 = []
pr_list_2 = []

for r in menu_name_list_1 :
    menu_1 = r[0]
    for name, price in zip(df_price['메뉴'], df_price['가격']) :
        if menu_1 == name :
            pr_list_1.append(price)
        
    try :
        menu_2 = r[1]
        for name_2, price_2 in zip(df_price['메뉴'], df_price['가격']) :
            if menu_2 == name_2 :
                pr_list_2.append(price_2)
                
    except :
        pr_list_2.append(0)
    

In [30]:
sum_list = []
for a, b in zip(pr_list_1, pr_list_2) :
    sum_list.append(a+b)
    
sum_list

[7500,
 8000,
 5000,
 7000,
 5000,
 7500,
 5000,
 1500,
 6500,
 1000,
 1000,
 7000,
 9000,
 1000,
 28000,
 1000,
 28000,
 1000,
 26000,
 1000,
 23000,
 1000,
 1500,
 1000,
 6500,
 7000,
 3000,
 7000,
 7000,
 7000,
 7000,
 7000,
 10000,
 9000,
 10000,
 4000,
 6500,
 9000,
 3000,
 6500,
 6500,
 18000,
 7000,
 6500,
 9000,
 4000,
 1500,
 9000,
 28000,
 4000,
 7000,
 3000,
 7000,
 4000,
 4000,
 9000,
 4000,
 4000,
 7000,
 4000,
 4000,
 7500,
 7500,
 7500,
 4000,
 32000,
 26000,
 4000,
 1500,
 9000,
 7500,
 9000,
 4000,
 18000,
 7000,
 18000,
 1500,
 18000,
 7000,
 1500,
 7500,
 12000,
 7500,
 1500,
 7500,
 7500,
 11500,
 6000,
 7500,
 5000,
 6500,
 1000,
 9000,
 12500,
 11500,
 5000,
 4000,
 7500,
 7500,
 13500,
 8000,
 6500,
 1000,
 7000,
 15500,
 7500,
 10000,
 6500,
 9000,
 1000,
 8000,
 6500,
 8000,
 7500,
 1500,
 1000,
 29000,
 32000,
 5000,
 28000,
 1000,
 4000,
 27000,
 30000,
 5000,
 26000,
 1000,
 4000,
 2500,
 10500,
 10000,
 1500,
 1000,
 9000,
 5500,
 5000,
 1500,
 1000,
 8500,

In [31]:
nov['ante_sum'] = sum_list
nov

,antecedents,consequents,support,confidence,lift,ante_sum
0,(짬뽕순두부국밥),(갈비시레기국밥),0.015306,0.166667,2.969697,7500
1,(갈비시레기국밥),(짬뽕순두부국밥),0.015306,0.272727,2.969697,8000
2,(고기만두),(내장국밥),0.010204,0.142857,1.750000,5000
3,(내장국밥),(고기만두),0.010204,0.125000,1.750000,7000
4,(고기만두),(모둠국밥),0.020408,0.285714,2.666667,5000
...,...,...,...,...,...,...
211,(무보까정식),"(짬뽕순두부국밥, 소주)",0.010204,0.052632,3.438596,9000
212,"(순대국밥, 소주)",(음료수),0.010204,0.166667,1.256410,11000
213,"(음료수, 소주)",(순대국밥),0.010204,0.333333,1.333333,5500
214,(순대국밥),"(음료수, 소주)",0.010204,0.040816,1.333333,7000


In [32]:
nov['consequents']

menu_name_list_2 = []

for q in nov['consequents'] :
    de = list(q)
    menu_name_list_2.append(de)
menu_name_list_2 = [i[0:] for i in menu_name_list_2]
menu_name_list_2

[['갈비시레기국밥'],
 ['짬뽕순두부국밥'],
 ['내장국밥'],
 ['고기만두'],
 ['모둠국밥'],
 ['고기만두'],
 ['음료수'],
 ['고기만두'],
 ['곱배기'],
 ['돼지국밥'],
 ['내장국밥'],
 ['공기밥'],
 ['공기밥'],
 ['떡갈비정식'],
 ['공기밥'],
 ['마늘보쌈(중)'],
 ['공기밥'],
 ['매콤이소갈비찜(중)'],
 ['공기밥'],
 ['무보까보쌈(중)'],
 ['공기밥'],
 ['순대곱창전골(중)'],
 ['공기밥'],
 ['음료수'],
 ['내장국밥'],
 ['돼지국밥'],
 ['내장국밥'],
 ['막걸리'],
 ['내장국밥'],
 ['맛보기순대'],
 ['내장국밥'],
 ['순대국밥'],
 ['무보까정식'],
 ['도가니탕'],
 ['소주'],
 ['도가니탕'],
 ['떡갈비정식'],
 ['돼지국밥'],
 ['돼지국밥'],
 ['막걸리'],
 ['수육(중)'],
 ['돼지국밥'],
 ['돼지국밥'],
 ['순대국밥'],
 ['소주'],
 ['떡갈비정식'],
 ['떡갈비정식'],
 ['음료수'],
 ['소주'],
 ['마늘보쌈(중)'],
 ['막걸리'],
 ['순대국밥'],
 ['소주'],
 ['맛보기순대'],
 ['무보까정식'],
 ['맥주'],
 ['소주'],
 ['맥주'],
 ['맥주'],
 ['순대국밥'],
 ['모둠국밥'],
 ['소주'],
 ['모둠국밥'],
 ['얼큰이국밥'],
 ['무보까보쌈(대)'],
 ['소주'],
 ['소주'],
 ['무보까보쌈(중)'],
 ['무보까정식'],
 ['음료수'],
 ['무보까정식'],
 ['짬뽕순두부국밥'],
 ['수육(중)'],
 ['소주'],
 ['수육(중)'],
 ['순대국밥'],
 ['수육(중)'],
 ['음료수'],
 ['음료수'],
 ['순대국밥'],
 ['순대한접시'],
 ['얼큰이국밥'],
 ['음료수'],
 ['얼큰이국밥'],
 ['짬뽕순두부국밥'],
 ['얼큰이국밥'],
 ['공기밥'],
 ['돼지국밥'],
 ['고기만두'],
 ['돼

In [33]:
pr_list_3 = []
pr_list_4 = []

for r in menu_name_list_2 :
    menu_1 = r[0]
    for name, price in zip(df_price['메뉴'], df_price['가격']) :
        if menu_1 == name :
            pr_list_3.append(price)
        
    try :
        menu_2 = r[1]
        for name_2, price_2 in zip(df_price['메뉴'], df_price['가격']) :
            if menu_2 == name_2 :
                pr_list_4.append(price_2)
                
    except :
        pr_list_4.append(0)
    

In [34]:
sum_list_2 = []
for a, b in zip(pr_list_3, pr_list_4) :
    sum_list_2.append(a+b)
    
sum_list_2

[8000,
 7500,
 7000,
 5000,
 7500,
 5000,
 1500,
 5000,
 1000,
 6500,
 7000,
 1000,
 1000,
 9000,
 1000,
 28000,
 1000,
 28000,
 1000,
 26000,
 1000,
 23000,
 1000,
 1500,
 7000,
 6500,
 7000,
 3000,
 7000,
 7000,
 7000,
 7000,
 9000,
 10000,
 4000,
 10000,
 9000,
 6500,
 6500,
 3000,
 18000,
 6500,
 6500,
 7000,
 4000,
 9000,
 9000,
 1500,
 4000,
 28000,
 3000,
 7000,
 4000,
 7000,
 9000,
 4000,
 4000,
 4000,
 4000,
 7000,
 7500,
 4000,
 7500,
 7500,
 32000,
 4000,
 4000,
 26000,
 9000,
 1500,
 9000,
 7500,
 18000,
 4000,
 18000,
 7000,
 18000,
 1500,
 1500,
 7000,
 12000,
 7500,
 1500,
 7500,
 7500,
 7500,
 1000,
 6500,
 5000,
 7500,
 6000,
 11500,
 7500,
 4000,
 5000,
 11500,
 12500,
 9000,
 7000,
 1000,
 6500,
 8000,
 13500,
 7500,
 1000,
 9000,
 6500,
 10000,
 7500,
 15500,
 6500,
 8000,
 1000,
 1500,
 7500,
 8000,
 4000,
 1000,
 28000,
 5000,
 32000,
 29000,
 4000,
 1000,
 26000,
 5000,
 30000,
 27000,
 9000,
 1000,
 1500,
 10000,
 10500,
 2500,
 1000,
 1500,
 5000,
 5500,
 1000,

In [35]:
nov['cons_sum'] = sum_list_2
nov

,antecedents,consequents,support,confidence,lift,ante_sum,cons_sum
0,(짬뽕순두부국밥),(갈비시레기국밥),0.015306,0.166667,2.969697,7500,8000
1,(갈비시레기국밥),(짬뽕순두부국밥),0.015306,0.272727,2.969697,8000,7500
2,(고기만두),(내장국밥),0.010204,0.142857,1.750000,5000,7000
3,(내장국밥),(고기만두),0.010204,0.125000,1.750000,7000,5000
4,(고기만두),(모둠국밥),0.020408,0.285714,2.666667,5000,7500
...,...,...,...,...,...,...,...
211,(무보까정식),"(짬뽕순두부국밥, 소주)",0.010204,0.052632,3.438596,9000,11500
212,"(순대국밥, 소주)",(음료수),0.010204,0.166667,1.256410,11000,1500
213,"(음료수, 소주)",(순대국밥),0.010204,0.333333,1.333333,5500,7000
214,(순대국밥),"(음료수, 소주)",0.010204,0.040816,1.333333,7000,5500


In [36]:
total_sum = []
for ante, cons in zip(nov['ante_sum'], nov['cons_sum']) :
    total_sum.append(ante + cons)

nov['total_sum'] = total_sum
nov

,antecedents,consequents,support,confidence,lift,ante_sum,cons_sum,total_sum
0,(짬뽕순두부국밥),(갈비시레기국밥),0.015306,0.166667,2.969697,7500,8000,15500
1,(갈비시레기국밥),(짬뽕순두부국밥),0.015306,0.272727,2.969697,8000,7500,15500
2,(고기만두),(내장국밥),0.010204,0.142857,1.750000,5000,7000,12000
3,(내장국밥),(고기만두),0.010204,0.125000,1.750000,7000,5000,12000
4,(고기만두),(모둠국밥),0.020408,0.285714,2.666667,5000,7500,12500
...,...,...,...,...,...,...,...,...
211,(무보까정식),"(짬뽕순두부국밥, 소주)",0.010204,0.052632,3.438596,9000,11500,20500
212,"(순대국밥, 소주)",(음료수),0.010204,0.166667,1.256410,11000,1500,12500
213,"(음료수, 소주)",(순대국밥),0.010204,0.333333,1.333333,5500,7000,12500
214,(순대국밥),"(음료수, 소주)",0.010204,0.040816,1.333333,7000,5500,12500


In [52]:
total_income

61266500

In [64]:
income_rate = []
for sp, t in zip(nov['support'], nov['total_sum']) :
    income_rate.append(round(sp * total_income))
nov['income_rate'] = income_rate
nov

,antecedents,consequents,support,confidence,lift,ante_sum,cons_sum,total_sum,income_rate
0,(짬뽕순두부국밥),(갈비시레기국밥),0.015306,0.166667,2.969697,7500,8000,15500,937753
1,(갈비시레기국밥),(짬뽕순두부국밥),0.015306,0.272727,2.969697,8000,7500,15500,937753
2,(고기만두),(내장국밥),0.010204,0.142857,1.750000,5000,7000,12000,625168
3,(내장국밥),(고기만두),0.010204,0.125000,1.750000,7000,5000,12000,625168
4,(고기만두),(모둠국밥),0.020408,0.285714,2.666667,5000,7500,12500,1250337
...,...,...,...,...,...,...,...,...,...
211,(무보까정식),"(짬뽕순두부국밥, 소주)",0.010204,0.052632,3.438596,9000,11500,20500,625168
212,"(순대국밥, 소주)",(음료수),0.010204,0.166667,1.256410,11000,1500,12500,625168
213,"(음료수, 소주)",(순대국밥),0.010204,0.333333,1.333333,5500,7000,12500,625168
214,(순대국밥),"(음료수, 소주)",0.010204,0.040816,1.333333,7000,5500,12500,625168


In [75]:
nov['antecedents'] = menu_name_list_1
nov['consequents'] = menu_name_list_2
nov

,antecedents,consequents,support,confidence,lift,ante_sum,cons_sum,total_sum,income_rate
0,[짬뽕순두부국밥],[갈비시레기국밥],0.015306,0.166667,2.969697,7500,8000,15500,937753
1,[갈비시레기국밥],[짬뽕순두부국밥],0.015306,0.272727,2.969697,8000,7500,15500,937753
2,[고기만두],[내장국밥],0.010204,0.142857,1.750000,5000,7000,12000,625168
3,[내장국밥],[고기만두],0.010204,0.125000,1.750000,7000,5000,12000,625168
4,[고기만두],[모둠국밥],0.020408,0.285714,2.666667,5000,7500,12500,1250337
...,...,...,...,...,...,...,...,...,...
211,[무보까정식],"[짬뽕순두부국밥, 소주]",0.010204,0.052632,3.438596,9000,11500,20500,625168
212,"[순대국밥, 소주]",[음료수],0.010204,0.166667,1.256410,11000,1500,12500,625168
213,"[음료수, 소주]",[순대국밥],0.010204,0.333333,1.333333,5500,7000,12500,625168
214,[순대국밥],"[음료수, 소주]",0.010204,0.040816,1.333333,7000,5500,12500,625168


In [92]:
nov = nov.sort_values(['income_rate'], ascending = False)
nov = nov.reset_index(drop = True)
nov

,antecedents,consequents,support,confidence,lift,ante_sum,cons_sum,total_sum,income_rate
0,[돼지국밥],[순대국밥],0.158163,0.329787,1.319149,6500,7000,13500,9690110
1,[순대국밥],[돼지국밥],0.158163,0.632653,1.319149,7000,6500,13500,9690110
2,[공기밥],[음료수],0.051020,0.192308,1.449704,1000,1500,2500,3125842
3,[음료수],[공기밥],0.051020,0.384615,1.449704,1500,1000,2500,3125842
4,[돼지국밥],[내장국밥],0.040816,0.085106,1.042553,6500,7000,13500,2500673
...,...,...,...,...,...,...,...,...,...
211,[얼큰이국밥],"[돼지국밥, 음료수]",0.010204,0.080000,1.568000,7500,8000,15500,625168
212,"[돼지국밥, 음료수]",[얼큰이국밥],0.010204,0.200000,1.568000,8000,7500,15500,625168
213,[맥주],"[소주, 무보까정식]",0.010204,0.285714,5.090909,4000,13000,17000,625168
214,"[소주, 무보까정식]",[맥주],0.010204,0.181818,5.090909,13000,4000,17000,625168


In [96]:
df

,판매일자,영수증번호,판매시간,영수증별총매출,현금,카드,key,구분,메뉴코드,메뉴명,메뉴가격,메뉴수량,메뉴가격*수량,요일,시간분할
0,2019-11-01,01-0001,09:28:42,8500,0,8500,r1,판매,0002,순대국밥,7000,1,7000,금요일,3
1,2019-11-01,01-0001,09:28:42,8500,0,8500,r1,판매,0047,음료수,1500,1,1500,금요일,3
2,2019-11-01,01-0002,09:35:02,25500,0,25500,r2,판매,0002,순대국밥,7000,2,14000,금요일,3
3,2019-11-01,01-0002,09:35:02,25500,0,25500,r2,판매,0005,모둠국밥,7500,1,7500,금요일,3
4,2019-11-01,01-0002,09:35:02,25500,0,25500,r2,판매,0043,소주,4000,1,4000,금요일,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6375,2019-11-30,01-0133,09:06:03,42000,42000,0,r3354,판매,0043,소주,4000,4,16000,토요일,3
6376,2019-11-30,01-0133,09:06:03,42000,42000,0,r3354,판매,0047,음료수,1500,4,6000,토요일,3
6377,2019-11-30,01-0134,09:09:57,6500,0,6500,r3355,판매,0001,돼지국밥,6500,1,6500,토요일,3
6378,2019-11-30,01-0135,09:28:13,26000,0,26000,r3356,판매,0001,돼지국밥,6500,4,26000,토요일,3


In [94]:
nov.to_csv('1109_df.csv', encoding = 'cp949', index = False)

'C:\\Users\\alswo\\Desktop\\식파마\\BasketAnalysis\\data\\상인점'

In [79]:
menu_name_list_1

[['짬뽕순두부국밥'],
 ['갈비시레기국밥'],
 ['고기만두'],
 ['내장국밥'],
 ['고기만두'],
 ['모둠국밥'],
 ['고기만두'],
 ['음료수'],
 ['돼지국밥'],
 ['곱배기'],
 ['공기밥'],
 ['내장국밥'],
 ['떡갈비정식'],
 ['공기밥'],
 ['마늘보쌈(중)'],
 ['공기밥'],
 ['매콤이소갈비찜(중)'],
 ['공기밥'],
 ['무보까보쌈(중)'],
 ['공기밥'],
 ['순대곱창전골(중)'],
 ['공기밥'],
 ['음료수'],
 ['공기밥'],
 ['돼지국밥'],
 ['내장국밥'],
 ['막걸리'],
 ['내장국밥'],
 ['맛보기순대'],
 ['내장국밥'],
 ['순대국밥'],
 ['내장국밥'],
 ['도가니탕'],
 ['무보까정식'],
 ['도가니탕'],
 ['소주'],
 ['돼지국밥'],
 ['떡갈비정식'],
 ['막걸리'],
 ['돼지국밥'],
 ['돼지국밥'],
 ['수육(중)'],
 ['순대국밥'],
 ['돼지국밥'],
 ['떡갈비정식'],
 ['소주'],
 ['음료수'],
 ['떡갈비정식'],
 ['마늘보쌈(중)'],
 ['소주'],
 ['순대국밥'],
 ['막걸리'],
 ['맛보기순대'],
 ['소주'],
 ['맥주'],
 ['무보까정식'],
 ['맥주'],
 ['소주'],
 ['순대국밥'],
 ['맥주'],
 ['소주'],
 ['모둠국밥'],
 ['얼큰이국밥'],
 ['모둠국밥'],
 ['소주'],
 ['무보까보쌈(대)'],
 ['무보까보쌈(중)'],
 ['소주'],
 ['음료수'],
 ['무보까정식'],
 ['짬뽕순두부국밥'],
 ['무보까정식'],
 ['소주'],
 ['수육(중)'],
 ['순대국밥'],
 ['수육(중)'],
 ['음료수'],
 ['수육(중)'],
 ['순대국밥'],
 ['음료수'],
 ['얼큰이국밥'],
 ['순대한접시'],
 ['얼큰이국밥'],
 ['음료수'],
 ['얼큰이국밥'],
 ['짬뽕순두부국밥'],
 ['고기만두', '돼지국밥'],
 ['고기만두', '공기밥'],

In [67]:
menu_name_list_2

[['갈비시레기국밥'],
 ['짬뽕순두부국밥'],
 ['내장국밥'],
 ['고기만두'],
 ['모둠국밥'],
 ['고기만두'],
 ['음료수'],
 ['고기만두'],
 ['곱배기'],
 ['돼지국밥'],
 ['내장국밥'],
 ['공기밥'],
 ['공기밥'],
 ['떡갈비정식'],
 ['공기밥'],
 ['마늘보쌈(중)'],
 ['공기밥'],
 ['매콤이소갈비찜(중)'],
 ['공기밥'],
 ['무보까보쌈(중)'],
 ['공기밥'],
 ['순대곱창전골(중)'],
 ['공기밥'],
 ['음료수'],
 ['내장국밥'],
 ['돼지국밥'],
 ['내장국밥'],
 ['막걸리'],
 ['내장국밥'],
 ['맛보기순대'],
 ['내장국밥'],
 ['순대국밥'],
 ['무보까정식'],
 ['도가니탕'],
 ['소주'],
 ['도가니탕'],
 ['떡갈비정식'],
 ['돼지국밥'],
 ['돼지국밥'],
 ['막걸리'],
 ['수육(중)'],
 ['돼지국밥'],
 ['돼지국밥'],
 ['순대국밥'],
 ['소주'],
 ['떡갈비정식'],
 ['떡갈비정식'],
 ['음료수'],
 ['소주'],
 ['마늘보쌈(중)'],
 ['막걸리'],
 ['순대국밥'],
 ['소주'],
 ['맛보기순대'],
 ['무보까정식'],
 ['맥주'],
 ['소주'],
 ['맥주'],
 ['맥주'],
 ['순대국밥'],
 ['모둠국밥'],
 ['소주'],
 ['모둠국밥'],
 ['얼큰이국밥'],
 ['무보까보쌈(대)'],
 ['소주'],
 ['소주'],
 ['무보까보쌈(중)'],
 ['무보까정식'],
 ['음료수'],
 ['무보까정식'],
 ['짬뽕순두부국밥'],
 ['수육(중)'],
 ['소주'],
 ['수육(중)'],
 ['순대국밥'],
 ['수육(중)'],
 ['음료수'],
 ['음료수'],
 ['순대국밥'],
 ['순대한접시'],
 ['얼큰이국밥'],
 ['음료수'],
 ['얼큰이국밥'],
 ['짬뽕순두부국밥'],
 ['얼큰이국밥'],
 ['공기밥'],
 ['돼지국밥'],
 ['고기만두'],
 ['돼

In [51]:
for receipt_num, menu, to_price in zip(df['영수증번호'], df['메뉴명'], df['메뉴가격*수량']) :
    if receipt_num == receipt_num :
        globals()[receipt_num]
        
        
        #receipt_num + _list = []
        

receipt_num_list

KeyError: '01-0001'

In [ ]:
for val_name in association_day_val_list:
    print("="*20,val_name,"="*20)
    print(globals()[val_name])
    print('\n')

In [63]:
df_price

,메뉴,빈도수,가격
0,돼지국밥,176,6500
1,순대국밥,73,7000
2,무보까정식,69,9000
3,소주,63,4000
4,공기밥,53,1000
5,얼큰이국밥,31,7500
6,모둠국밥,29,7500
7,짬뽕순두부국밥,27,7500
8,음료수,27,1500
9,내장국밥,23,7000


In [27]:
os.chdir('C:\\Users\\alswo\\Desktop\\식파마\\BasketAnalysis\\data\\상인점\\')
df_price = pd.read_csv('상인점_빈도수_가격.csv', encoding = 'cp949')
df_price

,메뉴,빈도수,가격
0,돼지국밥,176,6500
1,순대국밥,73,7000
2,무보까정식,69,9000
3,소주,63,4000
4,공기밥,53,1000
5,얼큰이국밥,31,7500
6,모둠국밥,29,7500
7,짬뽕순두부국밥,27,7500
8,음료수,27,1500
9,내장국밥,23,7000


In [34]:
len(menu_name_list_1)

216

In [60]:
price_list_1 = []

for i,menu_name in enumerate(menu_name_list_1):
    df_temp = df_price[df_price['메뉴']==menu_name]
    temp_price = df_temp['가격'].iloc[0]
    add_price = df_temp['가격'].iloc[0]
    price_list_1.append(temp_price)
price_list_1

ValueError: Lengths must match to compare

In [30]:
nov['consequents']

menu_name_list_2 = []

for q in nov['consequents'] :
    de = list(q)
    menu_name_list_2.append(de)
menu_name_list_2 = [i[0:] for i in menu_name_list_2]
menu_name_list_2

[['짬뽕순두부국밥'],
 ['갈비시레기국밥'],
 ['내장국밥'],
 ['고기만두'],
 ['모둠국밥'],
 ['고기만두'],
 ['음료수'],
 ['고기만두'],
 ['돼지국밥'],
 ['곱배기'],
 ['공기밥'],
 ['내장국밥'],
 ['떡갈비정식'],
 ['공기밥'],
 ['마늘보쌈(중)'],
 ['공기밥'],
 ['공기밥'],
 ['매콤이소갈비찜(중)'],
 ['무보까보쌈(중)'],
 ['공기밥'],
 ['공기밥'],
 ['순대곱창전골(중)'],
 ['음료수'],
 ['공기밥'],
 ['돼지국밥'],
 ['내장국밥'],
 ['내장국밥'],
 ['막걸리'],
 ['맛보기순대'],
 ['내장국밥'],
 ['순대국밥'],
 ['내장국밥'],
 ['무보까정식'],
 ['도가니탕'],
 ['도가니탕'],
 ['소주'],
 ['떡갈비정식'],
 ['돼지국밥'],
 ['돼지국밥'],
 ['막걸리'],
 ['수육(중)'],
 ['돼지국밥'],
 ['순대국밥'],
 ['돼지국밥'],
 ['떡갈비정식'],
 ['소주'],
 ['떡갈비정식'],
 ['음료수'],
 ['마늘보쌈(중)'],
 ['소주'],
 ['순대국밥'],
 ['막걸리'],
 ['맛보기순대'],
 ['소주'],
 ['맥주'],
 ['무보까정식'],
 ['맥주'],
 ['소주'],
 ['순대국밥'],
 ['맥주'],
 ['모둠국밥'],
 ['소주'],
 ['얼큰이국밥'],
 ['모둠국밥'],
 ['무보까보쌈(대)'],
 ['소주'],
 ['무보까보쌈(중)'],
 ['소주'],
 ['음료수'],
 ['무보까정식'],
 ['짬뽕순두부국밥'],
 ['무보까정식'],
 ['수육(중)'],
 ['소주'],
 ['수육(중)'],
 ['순대국밥'],
 ['음료수'],
 ['수육(중)'],
 ['음료수'],
 ['순대국밥'],
 ['얼큰이국밥'],
 ['순대한접시'],
 ['얼큰이국밥'],
 ['음료수'],
 ['얼큰이국밥'],
 ['짬뽕순두부국밥'],
 ['공기밥'],
 ['돼지국밥'],
 ['고기만두'],
 ['공

In [31]:
price_list_2 = []

for i,menu_name in enumerate(menu_name_list_2):
    df_temp = df_price[df_price['메뉴']==menu_name]
    temp_price = df_temp['가격'].iloc[0]
    price_list_2.append(temp_price)
price_list_2

ValueError: Lengths must match to compare

In [165]:
nov['ante_price'] = price_list_1
nov['cons_price'] = price_list_2

sum_list = []
for an, co in zip(price_list_1, price_list_2) :
    sum_price = an + co
    sum_list.append(sum_price)

nov['sum'] = sum_list
nov

,antecedents,consequents,support,confidence,lift,ante_price,cons_price,sum
0,(짬뽕순두부국밥),(갈비시레기국밥),0.015306,0.166667,2.969697,7500,8000,15500
1,(갈비시레기국밥),(짬뽕순두부국밥),0.015306,0.272727,2.969697,8000,7500,15500
2,(고기만두),(내장국밥),0.010204,0.142857,1.750000,5000,7000,12000
3,(내장국밥),(고기만두),0.010204,0.125000,1.750000,7000,5000,12000
4,(고기만두),(모둠국밥),0.020408,0.285714,2.666667,5000,7500,12500
...,...,...,...,...,...,...,...,...
211,(무보까정식),"(짬뽕순두부국밥, 소주)",0.010204,0.052632,3.438596,9000,7500,16500
212,"(순대국밥, 소주)",(음료수),0.010204,0.166667,1.256410,7000,1500,8500
213,"(음료수, 소주)",(순대국밥),0.010204,0.333333,1.333333,1500,7000,8500
214,(순대국밥),"(음료수, 소주)",0.010204,0.040816,1.333333,7000,1500,8500


In [166]:
nov

,antecedents,consequents,support,confidence,lift,ante_price,cons_price,sum
0,(짬뽕순두부국밥),(갈비시레기국밥),0.015306,0.166667,2.969697,7500,8000,15500
1,(갈비시레기국밥),(짬뽕순두부국밥),0.015306,0.272727,2.969697,8000,7500,15500
2,(고기만두),(내장국밥),0.010204,0.142857,1.750000,5000,7000,12000
3,(내장국밥),(고기만두),0.010204,0.125000,1.750000,7000,5000,12000
4,(고기만두),(모둠국밥),0.020408,0.285714,2.666667,5000,7500,12500
...,...,...,...,...,...,...,...,...
211,(무보까정식),"(짬뽕순두부국밥, 소주)",0.010204,0.052632,3.438596,9000,7500,16500
212,"(순대국밥, 소주)",(음료수),0.010204,0.166667,1.256410,7000,1500,8500
213,"(음료수, 소주)",(순대국밥),0.010204,0.333333,1.333333,1500,7000,8500
214,(순대국밥),"(음료수, 소주)",0.010204,0.040816,1.333333,7000,1500,8500


In [98]:
nov_test = nov
nov_test

,antecedents,consequents,support,confidence,lift,ante_price,cons_price,sum
0,(짬뽕순두부국밥),(갈비시레기국밥),0.015306,0.166667,2.969697,7500,8000,15500
1,(갈비시레기국밥),(짬뽕순두부국밥),0.015306,0.272727,2.969697,8000,7500,15500
2,(고기만두),(내장국밥),0.010204,0.142857,1.750000,5000,7000,12000
3,(내장국밥),(고기만두),0.010204,0.125000,1.750000,7000,5000,12000
4,(고기만두),(모둠국밥),0.020408,0.285714,2.666667,5000,7500,12500
...,...,...,...,...,...,...,...,...
211,(무보까정식),"(짬뽕순두부국밥, 소주)",0.010204,0.052632,3.438596,9000,7500,16500
212,"(순대국밥, 소주)",(음료수),0.010204,0.166667,1.256410,7000,1500,8500
213,"(음료수, 소주)",(순대국밥),0.010204,0.333333,1.333333,1500,7000,8500
214,(순대국밥),"(음료수, 소주)",0.010204,0.040816,1.333333,7000,1500,8500


In [141]:
nov_test["antecedents"] = nov_test["antecedents"].apply(lambda x: ' '.join(list(x))).astype("unicode")
nov_test["antecedents"]

0      짬               ,                             ...
1      갈               ,                             ...
2      고               ,                             ...
3      내               ,                             ...
4      고               ,                             ...
                             ...                        
211    무               ,                             ...
212    순               ,                             ...
213    음               ,                             ...
214    순               ,                             ...
215    음               ,                             ...
Name: antecedents, Length: 216, dtype: object

In [146]:
m_list = []
for i in nov_test["antecedents"] :
    nm = i.replace(',','')
    nc = nm.replace(' ', '')
    m_list.append(nc)
    
for s in menu_name_list_1 :
    na = nc.split(s)
    print(na)

In [151]:
m_list[0].split(' ')


['짬뽕순두부', '']

In [247]:
nov.to_csv('합계_중간.csv', encoding = 'cp949', index = False)


## 


In [152]:
s = frozenset({'GRO73461'})
value, = s
value


'GRO73461'

In [167]:
nov["antecedents"]

0       (짬뽕순두부국밥)
1       (갈비시레기국밥)
2          (고기만두)
3          (내장국밥)
4          (고기만두)
          ...    
211       (무보까정식)
212    (순대국밥, 소주)
213     (음료수, 소주)
214        (순대국밥)
215         (음료수)
Name: antecedents, Length: 216, dtype: object

In [172]:
for i in nov["antecedents"] :
    values, = i
    print(values)

짬뽕순두부국밥
갈비시레기국밥
고기만두
내장국밥
고기만두
모둠국밥
고기만두
음료수
돼지국밥
곱배기
내장국밥
공기밥
떡갈비정식
공기밥
마늘보쌈(중)
공기밥
매콤이소갈비찜(중)
공기밥
무보까보쌈(중)
공기밥
순대곱창전골(중)
공기밥
음료수
공기밥
돼지국밥
내장국밥
내장국밥
막걸리
내장국밥
맛보기순대
순대국밥
내장국밥
도가니탕
무보까정식
도가니탕
소주
돼지국밥
떡갈비정식
돼지국밥
막걸리
돼지국밥
수육(중)
순대국밥
돼지국밥
소주
떡갈비정식
음료수
떡갈비정식
마늘보쌈(중)
소주
순대국밥
막걸리
소주
맛보기순대
무보까정식
맥주
소주
맥주
순대국밥
맥주
모둠국밥
소주
얼큰이국밥
모둠국밥
소주
무보까보쌈(대)
소주
무보까보쌈(중)
음료수
무보까정식
짬뽕순두부국밥
무보까정식
수육(중)
소주
순대국밥
수육(중)
음료수
수육(중)
순대국밥
음료수
얼큰이국밥
순대한접시
음료수
얼큰이국밥
짬뽕순두부국밥
얼큰이국밥


ValueError: too many values to unpack (expected 1)

In [ ]:
price_list = []
for num, menu in enumerate(df_price['메뉴']) :
    if menu


In [212]:
price_list = []
for num, 
for num, name, menu in zip(range(0, len(menu_name_list_1)), menu_name_list_1, df_price['메뉴'].to_list) :
    if name == menu :
        price_list.append(df_price.iloc['가격'][num])
        
        

TypeError: zip argument #3 must support iteration

In [ ]:
len(menu_name_list_1)

In [207]:
df_price

,메뉴,빈도수,가격
0,돼지국밥,176,6500
1,순대국밥,73,7000
2,무보까정식,69,9000
3,소주,63,4000
4,공기밥,53,1000
5,얼큰이국밥,31,7500
6,모둠국밥,29,7500
7,짬뽕순두부국밥,27,7500
8,음료수,27,1500
9,내장국밥,23,7000


In [206]:
nov['consequents']

menu_name_list_2 = []

for q in nov['consequents'] :
    de = list(q)
    menu_name_list_2.append(de)
menu_name_list_2 = [i[0] for i in menu_name_list_2]
menu_name_list_2

['갈비시레기국밥',
 '짬뽕순두부국밥',
 '내장국밥',
 '고기만두',
 '모둠국밥',
 '고기만두',
 '음료수',
 '고기만두',
 '곱배기',
 '돼지국밥',
 '내장국밥',
 '공기밥',
 '떡갈비정식',
 '공기밥',
 '마늘보쌈(중)',
 '공기밥',
 '매콤이소갈비찜(중)',
 '공기밥',
 '무보까보쌈(중)',
 '공기밥',
 '공기밥',
 '순대곱창전골(중)',
 '음료수',
 '공기밥',
 '내장국밥',
 '돼지국밥',
 '내장국밥',
 '막걸리',
 '맛보기순대',
 '내장국밥',
 '내장국밥',
 '순대국밥',
 '도가니탕',
 '무보까정식',
 '소주',
 '도가니탕',
 '떡갈비정식',
 '돼지국밥',
 '막걸리',
 '돼지국밥',
 '돼지국밥',
 '수육(중)',
 '순대국밥',
 '돼지국밥',
 '떡갈비정식',
 '소주',
 '떡갈비정식',
 '음료수',
 '소주',
 '마늘보쌈(중)',
 '막걸리',
 '순대국밥',
 '맛보기순대',
 '소주',
 '무보까정식',
 '맥주',
 '소주',
 '맥주',
 '순대국밥',
 '맥주',
 '모둠국밥',
 '소주',
 '모둠국밥',
 '얼큰이국밥',
 '소주',
 '무보까보쌈(대)',
 '무보까보쌈(중)',
 '소주',
 '무보까정식',
 '음료수',
 '짬뽕순두부국밥',
 '무보까정식',
 '소주',
 '수육(중)',
 '순대국밥',
 '수육(중)',
 '음료수',
 '수육(중)',
 '순대국밥',
 '음료수',
 '순대한접시',
 '얼큰이국밥',
 '얼큰이국밥',
 '음료수',
 '얼큰이국밥',
 '짬뽕순두부국밥',
 '돼지국밥',
 '고기만두',
 '공기밥',
 '고기만두',
 '공기밥',
 '공기밥',
 '모둠국밥',
 '소주',
 '고기만두',
 '소주',
 '고기만두',
 '고기만두',
 '내장국밥',
 '돼지국밥',
 '공기밥',
 '돼지국밥',
 '공기밥',
 '공기밥',
 '떡갈비정식',
 '돼지국밥',
 '공기밥',
 '돼지국밥',
 '공기밥',
 '공기밥',
 '돼지

In [146]:
os.chdir('C:\\Users\\alswo\\Desktop\\식파마\\BasketAnalysis\\data\\상인점\\')
df_price = pd.read_csv('상인점_빈도수_가격.csv', encoding = 'cp949')
df_price

,메뉴,빈도수,가격
0,돼지국밥,176,6500
1,순대국밥,73,7000
2,무보까정식,69,9000
3,소주,63,4000
4,공기밥,53,1000
5,얼큰이국밥,31,7500
6,모둠국밥,29,7500
7,짬뽕순두부국밥,27,7500
8,음료수,27,1500
9,내장국밥,23,7000


In [62]:
## lift
# P(A∩B) / P(A) * P(B)

1. 지지도(Support)
전체 거래항목 중 상품 A와 상품 B를 동시에 포함하여 거래하는 비율을 의미하는데요,
A -> B 라고 하는 규칙이 전체 거래 중 차지하는 비율을 통해 해당 연관 규칙이 얼마나 의미가 있는 규칙인지를 확인합니다.

지지도 = P(A∩B)  :  A와 B가 동시에 포함된 거래 수 / 전체 거래 수


2. 신뢰도(Confidence)
상품 A를 포함하는 거래 중 A와 B가 동시에 거래되는 비중으로,
상품 A를 구매 했을 때 상품 B를 구매할 확률이 어느정도 되는지를 확인합니다.

신뢰도 =  P(A∩B) / P(A)  :  A와 B가 동시에 포함된 거래 수 / A가 포함된 거래 수
    
    
3. 향상도(Lift)
상품 A의 거래 중 항목 B가 포함된 거래의 비율 / 전체 상품 거래 중 상품 B가 거래된 비율
(A가 주어지지 않았을 때 B의 확률 대비 A가 주어졌을 때 B의 확률 증가 비율)

향상도 = P(A∩B) / P(A)*P(B)  = P (B|A) / P (B)

:  A와 B가 동시에 일어난 횟수 / A, B가 독립된 사건일 때 A,B가 동시에 일어날 확률

품목 A와 B사이에 아무런 관계가 상호 관계가 없으면 향상도는 1이고.  향상도가 1보다 높아질 수록 연관성이 높다고 할 수 있습니다. 이것은 또한 향상도가 1보다 크거나 작다면 우연적 기회(random chance)보다 우수하다고도 해석합니다.

## confidence
# A 가 발생 했을 때, B 가 발생할 확률
# Confidence: the probability that a transaction that contains
# the items on the left hand side of the rule (in our example, pencil and paper) 
# also contains the item on the right hand side (a rubber). 
# The higher the confidence, the greater the likelihood that the item on the right hand side will be purchased or, 
# in other words, the greater the return rate you can expect for a given rule.



In [63]:
association_day_val_list

['result_day_fal_mon',
 'result_day_fal_tue',
 'result_day_fal_wed',
 'result_day_fal_thu',
 'result_day_fal_fri',
 'result_day_fal_sat',
 'result_day_fal_sun']

## 요일별 support 값 확인

In [88]:
for val_name in val_list:
    print("="*20,val_name,"="*20)
    print(globals()[val_name])
    print('\n')

==================== itemsets_fal_mon ====================
      support              itemsets
0    0.056122             (갈비시레기국밥)
1    0.071429                (고기만두)
2    0.010204                 (곱배기)
3    0.265306                 (공기밥)
4    0.081633                (내장국밥)
5    0.020408                (도가니탕)
6    0.479592                (돼지국밥)
7    0.030612               (떡갈비정식)
8    0.030612             (마늘보쌈(중))
9    0.030612                 (막걸리)
10   0.030612               (맛보기순대)
11   0.025510          (매콤이소갈비찜(중))
12   0.035714                  (맥주)
13   0.107143                (모둠국밥)
14   0.010204            (무보까보쌈(대))
15   0.030612            (무보까보쌈(중))
16   0.193878               (무보까정식)
17   0.316327                  (소주)
18   0.035714               (수육(중))
19   0.015306           (순대곱창전골(중))
20   0.250000                (순대국밥)
21   0.010204               (순대한접시)
22   0.040816             (시레기들깨국밥)
23   0.127551               (얼큰이국밥)
24   0.132653                 (음료수)
25   

## 요일별 연관분석 결과

In [35]:
for val_name in association_day_val_list:
    print("="*20,val_name,"="*20)
    print(globals()[val_name])
    print('\n')
    
# 순서에 따라서 달라지는데 왜 달라지는지?
# 그 순서는 어디로부터 가져왔는지? 그냥 영수증에 찍힌 순서를 주문순서라고 보기는 힘들지 않나

# 보면 값들이 완전히 똑같은 것들이 몇개 있는데 값들이 정확하게 뽑아진게 맞는지? 0.288005 개많아

==================== result_day_spr_mon ====================
           antecedents        consequents  antecedent support  \
0            (마늘보쌈(중))              (공기밥)            0.028223   
1                (공기밥)          (마늘보쌈(중))            0.288005   
2                (공기밥)       (매콤이소갈비찜(대))            0.288005   
3         (매콤이소갈비찜(대))              (공기밥)            0.014753   
4                (공기밥)       (매콤이소갈비찜(중))            0.288005   
5         (매콤이소갈비찜(중))              (공기밥)            0.033355   
6                (공기밥)               (맥주)            0.288005   
7                 (맥주)              (공기밥)            0.081462   
8           (무보까보쌈(중))              (공기밥)            0.030148   
9                (공기밥)         (무보까보쌈(중))            0.288005   
10               (공기밥)        (순대곱창전골(중))            0.288005   
11         (순대곱창전골(중))              (공기밥)            0.033996   
12                (육수)              (공기밥)            0.099423   
13               (공기밥)       

          antecedents        consequents  antecedent support  \
0           (마늘보쌈(중))              (공기밥)            0.033133   
1               (공기밥)          (마늘보쌈(중))            0.268503   
2               (공기밥)       (매콤이소갈비찜(대))            0.268503   
3        (매콤이소갈비찜(대))              (공기밥)            0.014200   
4               (공기밥)       (매콤이소갈비찜(중))            0.268503   
5        (매콤이소갈비찜(중))              (공기밥)            0.029690   
6          (무보까보쌈(중))              (공기밥)            0.033133   
7               (공기밥)         (무보까보쌈(중))            0.268503   
8               (공기밥)       (사골부대찌개(1인))            0.268503   
9        (사골부대찌개(1인))              (공기밥)            0.014630   
10              (공기밥)        (순대곱창전골(중))            0.268503   
11        (순대곱창전골(중))              (공기밥)            0.037005   
12               (육수)              (공기밥)            0.082616   
13              (공기밥)               (육수)            0.268503   
14              (공기밥)         (콜라 500ml)

159            0.019753  0.013169    0.027211  1.377551  0.003609    1.007666  


==================== result_day_sum_sun ====================
           antecedents        consequents  antecedent support  \
0            (갈비시레기국밥)            (무보까정식)            0.080225   
1              (무보까정식)          (갈비시레기국밥)            0.156982   
2              (얼큰이국밥)          (갈비시레기국밥)            0.161752   
3            (갈비시레기국밥)            (얼큰이국밥)            0.080225   
4            (마늘보쌈(중))              (공기밥)            0.033825   
5                (공기밥)          (마늘보쌈(중))            0.284909   
6                (공기밥)       (매콤이소갈비찜(중))            0.284909   
7         (매콤이소갈비찜(중))              (공기밥)            0.028187   
8           (무보까보쌈(중))              (공기밥)            0.036860   
9                (공기밥)         (무보까보쌈(중))            0.284909   
10             (수육(중))              (공기밥)            0.039029   
11               (공기밥)            (수육(중))            0.284909   
12          

115            0.020389  0.014365    0.029412  1.442513  0.004407    1.009296  


==================== result_day_fal_fri ====================
               antecedents            consequents  antecedent support  \
0                   (고기만두)                 (돼지국밥)            0.045245   
1                   (돼지국밥)                 (고기만두)            0.338190   
2                (마늘보쌈(중))                  (공기밥)            0.031442   
3                    (공기밥)              (마늘보쌈(중))            0.283742   
4                    (공기밥)           (매콤이소갈비찜(대))            0.283742   
5             (매콤이소갈비찜(대))                  (공기밥)            0.016104   
6                    (공기밥)           (매콤이소갈비찜(중))            0.283742   
7             (매콤이소갈비찜(중))                  (공기밥)            0.041411   
8                     (맥주)                  (공기밥)            0.086273   
9                    (공기밥)                   (맥주)            0.283742   
10              (무보까보쌈(중))                  (공기밥)     

           antecedents        consequents  antecedent support  \
0            (갈비시레기국밥)          (시레기들깨국밥)            0.080489   
1            (시레기들깨국밥)          (갈비시레기국밥)            0.051452   
2               (고기만두)              (공기밥)            0.037188   
3                (공기밥)             (고기만두)            0.285278   
4               (고기만두)             (돼지국밥)            0.037188   
5               (돼지국밥)             (고기만두)            0.372389   
6            (마늘보쌈(중))              (공기밥)            0.033622   
7                (공기밥)          (마늘보쌈(중))            0.285278   
8                (공기밥)       (매콤이소갈비찜(대))            0.285278   
9         (매콤이소갈비찜(대))              (공기밥)            0.012226   
10               (공기밥)       (매콤이소갈비찜(중))            0.285278   
11        (매콤이소갈비찜(중))              (공기밥)            0.036169   
12          (무보까보쌈(중))              (공기밥)            0.031075   
13               (공기밥)         (무보까보쌈(중))            0.285278   
14             (수육(중))   

In [55]:
association_day_val_list

['result_day_spr_mon',
 'result_day_spr_tue',
 'result_day_spr_wed',
 'result_day_spr_thu',
 'result_day_spr_fri',
 'result_day_spr_sat',
 'result_day_spr_sun',
 'result_day_sum_mon',
 'result_day_sum_tue',
 'result_day_sum_wed',
 'result_day_sum_thu',
 'result_day_sum_fri',
 'result_day_sum_sat',
 'result_day_sum_sun',
 'result_day_fal_mon',
 'result_day_fal_tue',
 'result_day_fal_wed',
 'result_day_fal_thu',
 'result_day_fal_fri',
 'result_day_fal_sat',
 'result_day_fal_sun',
 'result_day_win_mon',
 'result_day_win_tue',
 'result_day_win_wed',
 'result_day_win_thu',
 'result_day_win_fri',
 'result_day_win_sat',
 'result_day_win_sun']

In [56]:
for val_name in association_day_val_list:
    idx_list=[]
    
    globals()[val_name]['index']=globals()[val_name].index

    for i in range(0,len(globals()[val_name])):
        ancd = globals()[val_name].iloc[i]['antecedents']
        consq = globals()[val_name].iloc[i]['consequents']
    
        if (('돼지국밥' in ancd) or ('물냉면' in ancd)) and (('물냉면' in consq) or ('돼지국밥' in consq)):
            idx_list.append(i)
            
        globals()[val_name]['result']=globals()[val_name]['index'].apply(lambda x: True if x in idx_list else False)

result_day_spr_mon
result_day_spr_tue
result_day_spr_wed
result_day_spr_thu
result_day_spr_fri
result_day_spr_sat
result_day_spr_sun
result_day_sum_mon
result_day_sum_tue
result_day_sum_wed
result_day_sum_thu
result_day_sum_fri
result_day_sum_sat
result_day_sum_sun
result_day_fal_mon
result_day_fal_tue
result_day_fal_wed
result_day_fal_thu
result_day_fal_fri
result_day_fal_sat
result_day_fal_sun
result_day_win_mon
result_day_win_tue
result_day_win_wed
result_day_win_thu
result_day_win_fri
result_day_win_sat
result_day_win_sun


In [57]:
##
print(val_name)
globals()[val_name]

result_day_win_sun


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,index,result
0,(마늘보쌈(중)),(공기밥),0.031486,0.291352,0.013434,0.426667,1.464438,0.004261,1.236014,0,False
1,(공기밥),(마늘보쌈(중)),0.291352,0.031486,0.013434,0.046110,1.464438,0.004261,1.015330,1,False
2,(공기밥),(매콤이소갈비찜(중)),0.291352,0.031066,0.019731,0.067723,2.179959,0.010680,1.039320,2,False
3,(매콤이소갈비찜(중)),(공기밥),0.031066,0.291352,0.019731,0.635135,2.179959,0.010680,1.942221,3,False
4,(공기밥),(맥주),0.291352,0.060034,0.020151,0.069164,1.152093,0.002660,1.009809,4,False
5,(맥주),(공기밥),0.060034,0.291352,0.020151,0.335664,1.152093,0.002660,1.066702,5,False
6,(무보까보쌈(중)),(공기밥),0.033585,0.291352,0.013014,0.387500,1.330007,0.003229,1.156977,6,False
7,(공기밥),(무보까보쌈(중)),0.291352,0.033585,0.013014,0.044669,1.330007,0.003229,1.011602,7,False
8,(수육(중)),(공기밥),0.040302,0.291352,0.011755,0.291667,1.001081,0.000013,1.000445,8,False
9,(공기밥),(수육(중)),0.291352,0.040302,0.011755,0.040346,1.001081,0.000013,1.000045,9,False


In [39]:
result_day_fal_mon.sort_values(by='lift',ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,index,result
75,(매콤이소갈비찜(중)),"(육수, 공기밥)",0.033284,0.051167,0.011426,0.343284,6.709028,0.009723,1.444813,75,False
70,"(육수, 공기밥)",(매콤이소갈비찜(중)),0.051167,0.033284,0.011426,0.223301,6.709028,0.009723,1.244647,70,False
108,"(소주, 매콤이소갈비찜(중))",(육수),0.021858,0.095877,0.012419,0.568182,5.926166,0.010324,2.093759,108,False
109,(육수),"(소주, 매콤이소갈비찜(중))",0.095877,0.021858,0.012419,0.129534,5.926166,0.010324,1.123699,109,False
73,(육수),"(공기밥, 매콤이소갈비찜(중))",0.095877,0.020864,0.011426,0.119171,5.711695,0.009425,1.111607,73,False
72,"(공기밥, 매콤이소갈비찜(중))",(육수),0.020864,0.095877,0.011426,0.547619,5.711695,0.009425,1.998588,72,False
127,(육수),"(수육(중), 소주)",0.095877,0.032290,0.017387,0.181347,5.616182,0.014291,1.182076,127,False
126,"(수육(중), 소주)",(육수),0.032290,0.095877,0.017387,0.538462,5.616182,0.014291,1.958934,126,False
110,(매콤이소갈비찜(중)),"(육수, 소주)",0.033284,0.071038,0.012419,0.373134,5.252583,0.010055,1.481915,110,False
107,"(육수, 소주)",(매콤이소갈비찜(중)),0.071038,0.033284,0.012419,0.174825,5.252583,0.010055,1.171529,107,False


In [40]:
part_val_list=[]

te = TransactionEncoder()

for season in season_list:

    for part in range(0,8):

        te_result = te.fit(season_dict[season][part]).transform(season_dict[season][part])

        te_df = pd.DataFrame(te_result, columns=te.columns_)

        globals()['itemsets_{}_part{}'.format(season[:3],str(part))]=apriori(te_df, min_support=0.01, use_colnames=True)

        part_val_list.append('itemsets_{}_part{}'.format(season[:3],str(part)))

In [41]:
part_val_list

['itemsets_spr_part0',
 'itemsets_spr_part1',
 'itemsets_spr_part2',
 'itemsets_spr_part3',
 'itemsets_spr_part4',
 'itemsets_spr_part5',
 'itemsets_spr_part6',
 'itemsets_spr_part7',
 'itemsets_sum_part0',
 'itemsets_sum_part1',
 'itemsets_sum_part2',
 'itemsets_sum_part3',
 'itemsets_sum_part4',
 'itemsets_sum_part5',
 'itemsets_sum_part6',
 'itemsets_sum_part7',
 'itemsets_fal_part0',
 'itemsets_fal_part1',
 'itemsets_fal_part2',
 'itemsets_fal_part3',
 'itemsets_fal_part4',
 'itemsets_fal_part5',
 'itemsets_fal_part6',
 'itemsets_fal_part7',
 'itemsets_win_part0',
 'itemsets_win_part1',
 'itemsets_win_part2',
 'itemsets_win_part3',
 'itemsets_win_part4',
 'itemsets_win_part5',
 'itemsets_win_part6',
 'itemsets_win_part7']

In [42]:
association_part_val_list=[]

for val_name in part_val_list:
    globals()['result_part_{}'.format(str(val_name[-9:]))]=association_rules(globals()[val_name],metric="lift",min_threshold=1)
    
    association_part_val_list.append('result_part_{}'.format(str(val_name[-9:])))

In [43]:
association_part_val_list

['result_part_spr_part0',
 'result_part_spr_part1',
 'result_part_spr_part2',
 'result_part_spr_part3',
 'result_part_spr_part4',
 'result_part_spr_part5',
 'result_part_spr_part6',
 'result_part_spr_part7',
 'result_part_sum_part0',
 'result_part_sum_part1',
 'result_part_sum_part2',
 'result_part_sum_part3',
 'result_part_sum_part4',
 'result_part_sum_part5',
 'result_part_sum_part6',
 'result_part_sum_part7',
 'result_part_fal_part0',
 'result_part_fal_part1',
 'result_part_fal_part2',
 'result_part_fal_part3',
 'result_part_fal_part4',
 'result_part_fal_part5',
 'result_part_fal_part6',
 'result_part_fal_part7',
 'result_part_win_part0',
 'result_part_win_part1',
 'result_part_win_part2',
 'result_part_win_part3',
 'result_part_win_part4',
 'result_part_win_part5',
 'result_part_win_part6',
 'result_part_win_part7']

In [97]:
result_day_fal_mon

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(짬뽕순두부국밥),(갈비시레기국밥),0.091837,0.056122,0.015306,0.166667,2.969697,0.010152,1.132653
1,(갈비시레기국밥),(짬뽕순두부국밥),0.056122,0.091837,0.015306,0.272727,2.969697,0.010152,1.248724
2,(고기만두),(내장국밥),0.071429,0.081633,0.010204,0.142857,1.750000,0.004373,1.071429
3,(내장국밥),(고기만두),0.081633,0.071429,0.010204,0.125000,1.750000,0.004373,1.061224
4,(고기만두),(모둠국밥),0.071429,0.107143,0.020408,0.285714,2.666667,0.012755,1.250000
...,...,...,...,...,...,...,...,...,...
211,(짬뽕순두부국밥),"(무보까정식, 소주)",0.091837,0.056122,0.010204,0.111111,1.979798,0.005050,1.061862
212,"(음료수, 소주)",(순대국밥),0.030612,0.250000,0.010204,0.333333,1.333333,0.002551,1.125000
213,"(순대국밥, 소주)",(음료수),0.061224,0.132653,0.010204,0.166667,1.256410,0.002082,1.040816
214,(음료수),"(순대국밥, 소주)",0.132653,0.061224,0.010204,0.076923,1.256410,0.002082,1.017007


In [98]:
monday = result_day_fal_mon

In [113]:
monday = monday.drop(columns=['conviction', 'leverage', 'antecedent support', 'consequent support'])

In [114]:
monday

,antecedents,consequents,support,confidence,lift
0,(짬뽕순두부국밥),(갈비시레기국밥),0.015306,0.166667,2.969697
1,(갈비시레기국밥),(짬뽕순두부국밥),0.015306,0.272727,2.969697
2,(고기만두),(내장국밥),0.010204,0.142857,1.750000
3,(내장국밥),(고기만두),0.010204,0.125000,1.750000
4,(고기만두),(모둠국밥),0.020408,0.285714,2.666667
...,...,...,...,...,...
211,(짬뽕순두부국밥),"(무보까정식, 소주)",0.010204,0.111111,1.979798
212,"(음료수, 소주)",(순대국밥),0.010204,0.333333,1.333333
213,"(순대국밥, 소주)",(음료수),0.010204,0.166667,1.256410
214,(음료수),"(순대국밥, 소주)",0.010204,0.076923,1.256410
